In [1]:
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import curve_fit
import ipywidgets as widgets

# read in the data

df_GDP = pd.read_csv('./summary_GDP.csv', header=0) #change csv name
df_GTP = pd.read_csv('./summary_GTP.csv', header=0)

timepoints = df_GDP['Deut Time (sec)'].unique()
peptides_GDP = df_GDP['Sequence'].unique()
peptides_GTP = df_GTP['Sequence'].unique()

df_peptides_resnum = df_GDP[["Start","End","Sequence"]]
df_peptides_resnum = df_peptides_resnum.drop_duplicates()
peptide_resnums = {}
for index, row in df_peptides_resnum.iterrows():
    peptide_resnums[ row['Sequence'] ] = row['Start'], row['End']

peptides_both = set(peptides_GDP).intersection(peptides_GTP)
peptides_one = set(peptides_GDP).difference(peptides_GTP)

In [2]:
dropdown_peptides = []
for peptide in peptides_both:
    peptide_start = str(peptide_resnums.get(peptide)[0])
    dropdown_peptides.append(peptide_start + '- ' + peptide)
dropdown_peptides = sorted(dropdown_peptides)

DDwidget = widgets.Dropdown(
    options = dropdown_peptides,
    value = '1- GSSAPAANGEVPTF',
    description = 'Peptide 1:',
    disabled = False,
)

DDwidget2 = widgets.Dropdown(
    options = dropdown_peptides,
    value = '108- HRDLVRVCENIPIVL',
    description = 'Peptide 2:',
    disabled = False,
)

display(DDwidget)
display(DDwidget2)

Dropdown(description='Peptide 1:', options=('1- GSSAPAANGEVPTF', '1- GSSAPAANGEVPTFKL', '1- GSSAPAANGEVPTFKLVL…

Dropdown(description='Peptide 2:', index=4, options=('1- GSSAPAANGEVPTF', '1- GSSAPAANGEVPTFKL', '1- GSSAPAANG…

In [3]:
peptide_input1 = str(DDwidget.value)
peptide_input1 = peptide_input1.split()[-1]

peptide_input2 = str(DDwidget2.value)
peptide_input2 = peptide_input2.split()[-1]


# for peptide in peptides_both:
#     print(peptide)
#     GDP_exchanged = []
#     GTP_exchanged = []
#     num_prolines = peptide[2:].count('P')
#     max_protons = len(peptide) - 2 - num_prolines

#     df_GDP_subset = df_GDP[["Sequence","Deut Time (sec)","#D"]]
#     df_GTP_subset = df_GTP[["Sequence","Deut Time (sec)","#D"]]
#     df_GDP_peptide = df_GDP_subset[df_GDP_subset["Sequence"] == peptide]
#     df_GTP_peptide = df_GTP_subset[df_GTP_subset["Sequence"] == peptide]
        
#     for timepoint in timepoints:
#         df_GDP_tp = df_GDP_peptide[df_GDP_peptide["Deut Time (sec)"] == timepoint]
#         GDP_exchanged.append(df_GDP_tp['#D'])
#         df_GTP_tp = df_GTP_peptide[df_GTP_peptide["Deut Time (sec)"] == timepoint]
#         GTP_exchanged.append(df_GTP_tp['#D'])
        
#     GDP_timepoints = []
#     GDP_index = 0
#     GDP_exchanged_ = []
#     for x in GDP_exchanged:
#         if not math.isnan(float(x)):
#             GDP_timepoints.append(timepoints[GDP_index])
#             GDP_exchanged_.append(float(x))
#         GDP_index = GDP_index + 1
        
#     GTP_timepoints = []
#     GTP_index = 0
#     GTP_exchanged_ = []
#     for x in GTP_exchanged:
#         if not math.isnan(float(x)):
#             GTP_timepoints.append(timepoints[GTP_index])
#             GTP_exchanged_.append(float(x))
#         GTP_index = GTP_index + 1

#     def exchange_fit(x, a, b, c, d, e, f, g):
#         max_protons = a + b + c + g
#         d > e
#         e > f
#         d > f
#         return max_protons - a * np.exp(-d * x) - b * np.exp(-e * x) - c * np.exp(-f * x) - g

#     if len(GDP_timepoints) > 3 and len(GTP_timepoints) > 3:

#         GDP_exchanged_list = []
#         for element in GDP_exchanged_:
#             GDP_exchanged_list.append(float(element))
#         GTP_exchanged_list = []
#         for element in GTP_exchanged_:
#             GTP_exchanged_list.append(float(element))

#         trialT = np.logspace(1, 4.5, 10000)
#         popt, pcov = curve_fit(f = exchange_fit, xdata = GDP_timepoints, ydata = GDP_exchanged_list,
#                                bounds = (0, [max_protons-1, max_protons-1, max_protons-1, 1, 1, 1, max_protons]),
#                                maxfev = 100000)
#         exchange_GDP = exchange_fit(trialT, *popt)
#         print("GDP")
#         print(popt)
#         perr = np.sqrt(np.diag(pcov))
        

#         trialT = np.logspace(1, 4.5, 10000)
#         popt, pcov = curve_fit(f = exchange_fit, xdata = GTP_timepoints, ydata = GTP_exchanged_list,
#                                bounds = (0, [max_protons-1, max_protons-1, max_protons-1, 1, 1, 1, max_protons]),
#                                maxfev = 100000)
#         exchange_GTP = exchange_fit(trialT, *popt)
#         print("GTP")
#         print(popt)
#         perr = np.sqrt(np.diag(pcov))

#         font = {'family' : 'DejaVu Sans',
#                 'weight' : 'normal',
#                 'size'   : 20
#                }
#         axes = {'titlesize' : 20,
#                 'titleweight' : 'bold',
#                 'labelsize' : 20
#                }

#         plt.rc('font', **font)
#         plt.rc('axes', **axes)
#         plt.rc('lines', lw = 3)

#         fig, ax = plt.subplots()
#         ax.plot(GDP_timepoints, GDP_exchanged_, 'o', color = 'yellowgreen', markersize = 12, label = 'GDP')
#         ax.plot(trialT, exchange_GDP, ls = '-', color = 'yellowgreen')
#         ax.plot(GTP_timepoints, GTP_exchanged_, 'o', markersize = 12, label = 'GTP', color = 'orange')
#         ax.plot(trialT, exchange_GTP, ls = '-', color = 'orange')
#         peptide_start = str(peptide_resnums.get(peptide)[0])
#         ax.set(xlabel = 'Time (s)', ylabel = '#D', title = peptide_start + ": " + peptide)
#         ax.set_ylim(-.25,len(peptide)-2)
#         ax.set_xscale('log')
#         ax.legend(frameon = True)
#         plt.savefig(peptide_start + "_" + peptide + ".png",  transparent=True)



FDVTSRITYKNVPNWHRDLVRV
YINAQCA
